## Desharnais

In [17]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection as ms
from sklearn.model_selection import GridSearchCV

## Preparando Dados

In [18]:
#Carregando dados

data, meta = arff.loadarff('datasets/desharnais.arff')
df = pd.DataFrame(data)
df.head()

,Project,TeamExp,ManagerExp,YearEnd,Length,Effort,Transactions,Entities,PointsAdjust,Envergure,PointsNonAjust,Language
0,1.0,1.0,4.0,85.0,12.0,5152.0,253.0,52.0,305.0,34.0,302.0,b'1'
1,2.0,0.0,0.0,86.0,4.0,5635.0,197.0,124.0,321.0,33.0,315.0,b'1'
2,3.0,4.0,4.0,85.0,1.0,805.0,40.0,60.0,100.0,18.0,83.0,b'1'
3,4.0,0.0,0.0,86.0,5.0,3829.0,200.0,119.0,319.0,30.0,303.0,b'1'
4,5.0,0.0,0.0,86.0,4.0,2149.0,140.0,94.0,234.0,24.0,208.0,b'1'


In [19]:
def one_hot_enconder(X_transform, transform_data): #X_transform = data, transform_data = colunas
    
    for col in transform_data:
        cols = X_transform[col].unique() #retorna os valores da categoria que serão transformados em colunas 
        col_position = X_transform.columns.get_loc(col) #retorna posição da coluna, a partir dessa posição será adicionado as demais
        col_values = X_transform.pop(col) #retorna os valores da coluna primordial e apaga e remove 
        for c in cols: 
            X_transform.insert(col_position, str(col) + "_" + str(c).replace('b\'','').replace('\'',''), col_values.eq(c))
            col_position += 1

    return X_transform.replace({True: 1, False: 0})  

In [20]:
#removendo atributos desnecessário "Project" (id do projeto), "PointsAdjust" (PoinstNonAjust + fator
#de ajustes na técnica de pontos por função) e YearEnd.

df = df.drop(['Project','YearEnd', 'PointsAdjust'], axis=1)

#removendo linhas com atributos nulos ou vazios

df = df.dropna()

#Convertendo atributos categóricos em colunas. "OneHotEncoder"

transform_data = ['Language']
df =  one_hot_enconder(df, transform_data)

#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

df = (df-df.mean())/df.std()

#separando atributo classe Effort para entrada nos modelos

X = df.drop(['Effort'], axis=1)
y = df['Effort'].values.ravel()

## Treinando e avaliando o desempenho dos modelos

In [104]:
cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
models = []
#Instanciando os modelos #0 - nome, 1 - instância, 2 - parâmetros
models.append([' DT', DecisionTreeRegressor(), { "criterion" : ["squared_error", "friedman_mse", "absolute_error"],
                                                "splitter": ["best", "random"],
                                                "max_depth": np.arange(4,12),
                                                "random_state": np.arange(2,6)
                                                }])
models.append(['KNN', KNeighborsRegressor(), {  "n_neighbors" : np.arange(1,10),
                                                "weights": ["uniform", "distance"],
                                                "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']
                                                }])
models.append(['MLP', MLPRegressor(), {"hidden_layer_sizes": np.arange(10,100,10),
                                                "activation": ['identity', 'logistic', 'tanh', 'relu'],
                                                "solver": ['lbfgs', 'sgd', 'adam'],
                                                "random_state": [0], #np.arange(0,6),  #fixado para reduzir o tempo de processamento.
                                                "max_iter": [1500]
                                                }])
models.append(['RFR', RandomForestRegressor(), {"n_estimators": np.arange(5,20,2),
                                                    "max_depth" : np.arange(5,20,2),
                                                    "random_state": np.arange(2,6),
                                                    }])
models.append(['SVR', svm.SVR(), { "kernel" : [ 'poly', 'linear', 'rbf', 'sigmoid'], 
                                        "C": np.arange(0.001,0.01, 0.001), 
                                        "epsilon": np.arange(0.0001,0.001,0.0001), 
                                    }])

In [107]:
for model in models:
    clf = GridSearchCV(model[1], model[2], scoring='neg_mean_absolute_error',  cv=cv,  n_jobs=-1)
    clf.fit(X, y)
    print(">", model[0], ">------ MAE: %.3f" % (1 + clf.best_score_), " ----- STD: %.3f" 
    % (clf.cv_results_['std_test_score'][clf.best_index_]), " ----- Best Params:", (clf.best_params_)) 

> DT >------ MAE: 0.513  ----- STD: 0.106  ----- Best Params: {'criterion': 'squared_error', 'max_depth': 11, 'random_state': 2, 'splitter': 'random'}
> KNN >------ MAE: 0.452  ----- STD: 0.070  ----- Best Params: {'algorithm': 'auto', 'n_neighbors': 4, 'weights': 'distance'}
> MLP >------ MAE: 0.551  ----- STD: 0.032  ----- Best Params: {'activation': 'relu', 'hidden_layer_sizes': 50, 'max_iter': 1500, 'random_state': 0, 'solver': 'sgd'}
> RFR >------ MAE: 0.478  ----- STD: 0.058  ----- Best Params: {'max_depth': 5, 'n_estimators': 11, 'random_state': 3}
> SVR >------ MAE: 0.543  ----- STD: 0.182  ----- Best Params: {'C': 0.009000000000000001, 'epsilon': 0.0008, 'kernel': 'linear'}
